# **Preparacion de datos**

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures # <-- NUEVO
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from scipy.stats import loguniform

# 1. CARGA DE DATOS Y PREPROCESAMIENTO (IDÉNTICO AL ANTERIOR)

df = pd.read_csv('brain_tumor_dataset.csv')
columns_to_drop = [
    'Patient_ID', 'Symptom_1', 'Symptom_2', 'Symptom_3',
    'Radiation_Treatment', 'Surgery_Performed',
    'Chemotherapy', 'Follow_Up_Required'
]
df = df.drop(columns=columns_to_drop)
le = LabelEncoder()
y = le.fit_transform(df['Tumor_Type'])
X = df.drop(columns=['Tumor_Type'])
categorical_cols = ['Gender', 'Location', 'Histology', 'Stage', 'Family_History', 'MRI_Result']
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# 1. Crear características polinomiales de grado 2 (interacciones y cuadrados)
# Esto transforma el problema no lineal en un problema lineal de mayor dimensión.
poly = PolynomialFeatures(degree=2, include_bias=False)
# Usamos .values para aplicar la transformación, y luego convertimos a DataFrame temporalmente
X_poly_array = poly.fit_transform(X)
X_poly = pd.DataFrame(X_poly_array, columns=poly.get_feature_names_out(X.columns))

# 2. Escalado de TODAS las características (ahora incluyendo las polinomiales)
scaler = StandardScaler()
X_array = scaler.fit_transform(X_poly)
y_array = y

# **Optimizacion Bayesiana con BayesSearchCV**

In [15]:
# 1. Modelo Base RÁPIDO: LinearSVC
base_svc = LinearSVC(
    random_state=42,
    dual=False,
    max_iter=10000
)

# 2. Envoltorio para Probabilidades: CalibratedClassifierCV
calibrated_svc = CalibratedClassifierCV(
    estimator=base_svc,
    method='sigmoid',
    cv=5
)

# 3. Espacio de Búsqueda para C
param_distributions = {
    'estimator__C': loguniform(1e-3, 1e3)
}

# 4. Configurar CV K-Fold (K=10)
cv_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 5. Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=calibrated_svc,
    param_distributions=param_distributions,
    n_iter=80,
    scoring='roc_auc',
    cv=cv_kfold,
    n_jobs=-1,
    verbose=0,
    random_state=42
)

# 6. Ejecutar la Búsqueda
print("Iniciando la Búsqueda Aleatoria con SVC Lineal y Características Polinomiales...")
random_search.fit(X_array, y_array)
print("Búsqueda completada.")

Iniciando la Búsqueda Aleatoria con SVC Lineal y Características Polinomiales...
Búsqueda completada.


# **Resultados**

In [16]:
best_auc = random_search.best_score_
best_C = random_search.best_params_['estimator__C']

print("\n" + "="*50)
print("RESULTADO FINAL CON INGENIERÍA DE CARACTERÍSTICAS")
print(f"Mejor AUC promedio (CV K=10): {best_auc:.4f}")
print(f"Mejor valor de C (optimizado): {best_C:.4f}")
print("="*50)


RESULTADO FINAL CON INGENIERÍA DE CARACTERÍSTICAS
Mejor AUC promedio (CV K=10): 0.5009
Mejor valor de C (optimizado): 0.0011
